In [29]:
import pandas as pd
import geopandas as gpd

In [30]:
def describir_shp(gdf):
    area_cargado = gdf['area'].sum()
    gdf['area'] = gdf.geometry.area / 10000
    area_recalculado = gdf['area'].sum()
    gdf['idd'] = gdf.apply(lambda row: f"{row['unidad_01']}|{row['unidad_03']}|{row['unidad_05']}", axis=1)
    gdf_disolve = gdf.dissolve(by='idd')
    gdf_disolve['area'] = gdf_disolve.geometry.area / 10000
    area_disolve = gdf_disolve['area'].sum()
    estado_datos = {
        'Items' : ['Area cargado', 'Area recalculado', 'Area disolve'],
        'Datos' : [round(area_cargado, 2), round(area_recalculado, 2), round(area_disolve, 2)]
    }
    return pd.DataFrame(estado_datos)

In [31]:
path_shp_cosecha = r'C:/Documents/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - CATASTRO/Catastro/CATASTRO 2024/cosecha2024.shp'
path_shp_catastro = r'C:/Documents/OneDrive - Ingenio Azucarero Guabira S.A/INFORMACION GENERAL/CATASTRO/NUEVO/catastro030624.shp'

In [32]:
gdf_cosecha = gpd.read_file(path_shp_cosecha)
gdf_catastro = gpd.read_file(path_shp_catastro)

In [33]:
describir_shp(gdf_cosecha)

,Items,Datos
0,Area cargado,54111.39
1,Area recalculado,54091.11
2,Area disolve,54091.11


In [34]:
gdf_catastro_canha = gdf_catastro[gdf_catastro['cultivo']=='canha'].copy()

In [35]:
describir_shp(gdf_catastro_canha)

,Items,Datos
0,Area cargado,54364.74
1,Area recalculado,54365.04
2,Area disolve,54365.04


In [36]:
def comprobar_idd_repetidos(gdf):
    gdf['idd'] = gdf.apply(lambda row: f"{row['unidad_01']}|{row['unidad_03']}|{row['unidad_05']}", axis=1)
    gdf_orden = gdf.sort_values(by='idd')
    conteos = gdf['idd'].value_counts()
    valores_repetidos = conteos[conteos > 1]
    return valores_repetidos

# COMPROBAR CATASTRO

In [37]:
comprobar_idd_repetidos(gdf_catastro_canha)

idd
119|2042|L2.1    2
119|2042|L2.2    2
Name: count, dtype: int64

In [38]:
gdf_catastro_disolve = gdf_catastro_canha.dissolve(by='idd')
gdf_catastro_disolve['area'] = gdf_catastro_disolve.geometry.area / 10000
gdf_catastro_disolve = gdf_catastro_disolve.reset_index()
area_catastro_disolve = gdf_catastro_disolve['area'].sum()

In [39]:
area_catastro_disolve

54365.03798392745

In [40]:
comprobar_idd_repetidos(gdf_catastro_disolve)

Series([], Name: count, dtype: int64)

# COMPROBAR SHP COSECHA

In [41]:
gdf_cosecha_disolve = gdf_cosecha.dissolve(by='idd')
gdf_cosecha_disolve['area'] = gdf_cosecha_disolve.geometry.area / 10000
gdf_cosecha_disolve = gdf_cosecha_disolve.reset_index()
area_cosecha_disolve = gdf_cosecha_disolve['area'].sum()

In [42]:
area_cosecha_disolve

54091.110814872714

In [43]:
comprobar_idd_repetidos(gdf_cosecha_disolve)

Series([], Name: count, dtype: int64)

# COMPARAR CATASTRO - COSECHA

In [44]:
gdf_catastro_disolve['area'].sum()

54365.03798392745

In [45]:
gdf_cosecha_disolve['area'].sum()

54091.110814872714

In [46]:
gdf_catastro_disolve.columns

Index(['idd', 'geometry', 'id', 'unidad_01', 'unidad_02', 'unidad_03',
       'unidad_04', 'unidad_05', 'variedad', 'soca', 'zona', 'textura',
       'cultivo', 'zafra', 'financia', 'fs', 'fc', 'estado', 'area'],
      dtype='object')

In [47]:
gdf_cat_pivot = gdf_catastro_disolve[['idd', 'area']]

In [48]:
gdf_cos_pivot = gdf_cosecha_disolve[['idd', 'area']]

In [49]:
df_cat_merged = pd.merge(gdf_cat_pivot, gdf_cos_pivot, on='idd', how='left')

In [50]:
df_cat_merged.to_excel('merge.xlsx')